## 注意

このノートブックは**2018**年の西研教育のために作成されたものである。機械学習は移り変わりが激しい分野のためその年ごとに修正することをおすすめする（特にニューラルネットの項）。　2018年教育担当　中村太一

# 目次

1. 目的
2. 機械学習実践
    1. データを用意（タイタニック号）
    2. サポートベクトルマシン(SVM)
    3. ランダムフォレスト(RF)
    4. ニューラルネット(NN)
    5. アンサンブル学習
3. 学習器の特性
    1. データを用意（生成データ）
    1. サポートベクトルマシン(SVM)
    2. ランダムフォレスト(RF)
    3. ニューラルネット(NN)

# 1.目的

## 本講義では「機械学習を正しく使用する知識」の獲得を目的とします

さっそく実践！！

# 2.機械学習実践 

諸々の必要となるライブラリを読み込む

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

今回の講習で使うデータはタイタニック号の生存者/死亡者データ

## A.データを用意（タイタニック号）

<img src="https://upload.wikimedia.org/wikipedia/commons/6/6e/St%C3%B6wer_Titanic.jpg" width="480px">

タイタニック号の生存者/死亡者データを読み込み(https://www.kaggle.com/c/titanic/data)

In [ ]:
titanic_data = pd.read_csv('train.csv')
print(titanic_data.shape)
titanic_data.head()

このデータはタイタニック号の乗客の名前、性別、チケットのクラス等の乗客情報と沈没による生存/死亡の情報からなるデータで、乗客情報から生存/死亡を予測する学習器を構成したい

![](./MLfig.png)

このデータ中にあるSurvivedを他の情報から予測できる機械学習器（以降学習器）を作成したい。そのためにまずデータを整備する

使うデータ（'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'）の中で欠損値（NaN）を含むものを削除する

In [ ]:
titanic_data = titanic_data[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']].dropna(axis=0)
print(titanic_data.shape)

使うデータに絞ったこと、欠損値を削除した影響で表の大きさが小さくなっているのがわかる。

In [ ]:
titanic_data['Sex'].head()

'Sex'の項は文字列なのでこのままでは学習器に入力することができない。femaleを0、maleを1として数値に置き換える

In [ ]:
def sex_chikan(sex):
    if sex == 'female':
        return 0
    elif sex == 'male':
        return 1
    else:
        return sex
titanic_data['Sex'] = titanic_data['Sex'].apply(sex_chikan)
#titanic_data['Sex'] = titanic_data['Sex'].apply(lambda x:0 if x=='female' else 1) 
#上のようにlambda式を使っても処理できる。わざわざsex_chikanなんて作る必要がなくなる
titanic_data['Sex'].head()

学習器に対して入力にあたる乗客情報を取り出す。学習器では入力にあたるデータをｘとして表すことが多い

In [ ]:
x_data = titanic_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]
x_data.head()

適当な処理を施すことでIdや名前等の名義尺度を順序尺度にする
[名義尺度、順序尺度、間隔尺度、比率尺度](https://mathwords.net/syakudo)

学習器に対して出力にあたる生存/死亡を取り出す。ここでは学習器では出力にあたるデータをtとして表す

In [ ]:
t_data = titanic_data['Survived']
t_data.head()

そのままpandas.DataFrame型として扱いたいところだが、学習器に入力する上で都合が悪いので通常numpy.array型に変換する

In [ ]:
#numpy.array型で乗客のでデータと生存情報を置く
x_values = x_data.values
t_values = t_data.values

学習器を学習させるためのデータ（train）と学習器をテストするためのデータ（test）に分ける

ここで、「なぜ分ける必要があるのか？」と疑問に思うかもしれない。その疑問を解決するためには「過学習」「汎化性能」というキーワードと以下のTipsを参照してみると良い

### Tips
> 最小二乗法などを思い出して見ると、すべてのデータを使って関数を近似し、その残差をもって評価する．そのため，全てのデータを使ってしまっても構わないと思うかもしれない。しかし、一般的な学習器を考えるとき、学習に使われたデータにのみ正しい答えが出力できる学習器を構成してしまう可能性がある。例えば、ある入力に対して今までの入力が一致するものがあれば、その出力を答える学習器があれば、全てのデータを学習に使えば、それに対して正答率100％の学習器が構成できる。実際こういう方法で学習器を構成することもあるが、このような方法は入力データに大きく依存しすぎてしまい「過学習」と呼ばれる過度に入力データにフィッティングしたものになる。このような学習器は「汎化性能」と呼ばれる、これから入力するまだ得られていない入力データ（タイタニック号で言えばこれから沈没する船の乗客情報）を入力したときの性能が低くなる。汎化性能こそ、学習器に獲得してほしい能力なのでこれをテストするために学習するデータ（train）とテストするデータ（train）にデータを分割するのが普通である．

In [ ]:
#ランダムに分割するためのランダムなインデックスを用意
shuffle_index = np.arange(x_values.shape[0])
np.random.seed(1919) #乱数のシードを1919に固定（通常は要らない。講義の都合上必要）
np.random.shuffle(shuffle_index)

train_length = 500 #学習データの大きさ
x_train, x_test = x_values[shuffle_index[:train_length]], x_values[shuffle_index[train_length:]]
t_train, t_test = t_values[shuffle_index[:train_length]], t_values[shuffle_index[train_length:]]

ここまでで、学習器に使うデータ４種類(x_train, x_test, t_train, t_test)を全て用意できた。ここからは、学習器にtrainを使って学習させ、その性能をtestを使い見ていく。使う学習器はなかでも有名なサポートベクトルマシン（SVM）,ランダムフォレスト（RF）、ニューラルネット（NN）を使い。これらを比較して見よう

## B.サポートベクトルマシン（SVM）

さっそくSVMを使ってみよう

pythonで学習器を使うコードはものの数行で足りる。機械学習だからといって何も構える必要は一切ない

学習にはtrain以外使っていないことに注意しよう

In [ ]:
from sklearn.svm import SVC
svm = SVC(random_state=0)  # SVMの学習器を用意 random_stateは講義の都合上指定する必要があった
svm.fit(x_train, t_train)# 学習データを学習

実はさまざまなパラメータがデフォルトとして指定されている(あとで、すこし見ていく）

SVMの学習器の性能をtestを使って見ていく

In [ ]:
y = svm.predict(x_test)

#R  = sklearn.metrics.classification_report(y, t_test)
#print(R) 暇だったら見てみると面白い
a = sklearn.metrics.accuracy_score(t_test, y)
print('accuracy:', a)

accuracyがいわゆる精度で、どのくらい正解の出力を出せたかを表す。他にも様々な指標があるが、これは自分でコメントアウトを戻して確認してほしい

### Tips:
> - 機械学習関連を勉強しているとよくTrue Positive(TP)だとかFalse Negative(FN)という言葉を聞く。これは学習器がどう正解したか、どう間違ったかの様を表す言葉である。「True」は「正しい」ということで学習器が正しい出力を出したということ、逆に「False」は「間違った」出力を出したということだ。つぎに、「Positive」は「陽」のという意味で，これは学習するデータに依存する．例えばタイタニック号であれば，「生存する」という意味になる。「Negative」は逆に「陰」、「死亡する」という意味だ。ここまでの内容を総括すると、「True Positive」とは「正しく生存する」と答えたということで、「False Negative」とは「間違って死亡した」と答えたということだ。つまり、どちらも正しくは生存するが答えなのだが、「正しく生存する」と答え、他方は「間違って死亡する」と答えた。ただそれだけのことだ。聞いてみれば当たり前のことだが、こういう話は基礎的で引っかかりやすいし、話も単純なので酒の席なのでよく重宝される豆知識である。
> -  [wikipedia precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall)

> $Precision =  \frac{TP}{TP + FP} \\
Recall =  \frac{TP}{TP + FN} \\
F =  2\frac{Precision*Recall}{Precision + Recall}$ 

ここまでで、一連の使い方は終わり。データさえ用意してしまえば、大した労力をいとわないことがわかるだろう

## C.ランダムフォレスト（RF）

SVM同様、RFを使ってみる。RFもほとんど同じように使える

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)  # RFの学習器を用意
rf.fit(x_train, t_train)# 学習データを学習

In [ ]:
y = rf.predict(x_test)

#R  = sklearn.metrics.classification_report(y, t_test)
#print(R) 暇だったら見てみると面白い
a = sklearn.metrics.accuracy_score(t_test, y)
print('accuracy:', a)

使う学習器の名前が変わっただけで、ほとんど他には変化がない。このようにライブラリさえ使ってしまえば、たいていの機械学習器は簡単に構成できる

## D.ニューラルネット（N.N）

最近流行りのNNを使ってみよう。NNの利用法としてあまり一般的な方法ではないが簡単のためsklearnを使う

In [ ]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(random_state=0)
nn.fit(x_train, t_train)

In [ ]:
y = nn.predict(x_test)

#R  = sklearn.metrics.classification_report(y, t_test)
#print(R) 暇だったら見てみると面白い
a = sklearn.metrics.accuracy_score(t_test, y)
print('accuracy:', a)

ここまでで3つの学習器を見てきたが、精度が異なっていることが分かるだろう

さて、精度に差があることが分かるだろう。この「差」に注目して、さらに良い学習器を作っていきたい

## E.アンサンブル学習

[参考になるページ](https://mlwave.com/kaggle-ensembling-guide/)

「３人集まれば文殊の知恵」ということわざにもあるように、学習器も集めればより良いものが作れそうだ。このやり方を**アンサンブル学習**と言う

まずはそれぞれの学習器の出力をリストとして集める

In [ ]:
clf_list = [svm, rf, nn]
y_list = []
for clf in clf_list:
    y_list.append(clf.predict(x_test))
y_list = np.array(y_list)

今回は「集め方」として多数決を使う

In [ ]:
y = (y_list.sum(axis=0) > 1).astype(np.int) #３つの学習器の多数決を取る
#R  = sklearn.metrics.classification_report(y, t_test)
#print(R)
a = sklearn.metrics.accuracy_score(t_test, y)
print('accuracy:', a)

今回は良くなったとはいえない結果だった

この結果の理由を学習器の差、特に相関に注目して考察してみよう

それぞれの学習器判定の関係性について見てみるための関数を定義する（詳細を眺める必要はない）

In [ ]:
def kuraberu(a, b, name_a, name_b):
    result = []
    print(name_b+'|'+name_a+'\t0\t1')
    print('0\t{}\t{}\t{}'.format((b[a==0]==0).sum(), (b[a==1]==0).sum(), (b==0).sum()))
    print('1\t{}\t{}\t{}'.format((b[a==0]==1).sum(), (b[a==1]==1).sum(), (b==1).sum()))
    print('\t{}\t{}\t{}'.format((a==0).sum(), (a==1).sum(), a.shape[0]))

SVM と RFの相関

In [ ]:
kuraberu(y_list[0], y_list[1], 'svm', 'rf')
print('相関', np.corrcoef(y_list[0], y_list[1])[0,1])

SVM と NNの相関

In [ ]:
kuraberu(y_list[0], y_list[2], 'svm', 'nn') 
print('相関', np.corrcoef(y_list[0], y_list[2])[0,1])

RF と NNの相関

In [ ]:
kuraberu(y_list[1], y_list[2], 'rf', 'nn')
print('相関', np.corrcoef(y_list[1], y_list[2])[0,1])

この相関はランダムに与えた値によって大きく変化するため一般的に言える相関がここに現れているわけではないことに注意

相関は-1～1を取る値で絶対値が大きいほど相関が大きい。よって、全ての学習器において、そこそこ大きな相関があることが分かる。実はこれはアンサンブル学習にはあまり好ましくない

極端なことを言えば、全ての学習器の相関が1、つまり全く同じ出力をするときに、多数決を取ることに全く意味がないことが分かるだろう。アンサンブル学習に使う学習器はできるだけ独立な方がよいのだ

また、多数決という方法もあまり使われない。「Stacked Generalization & Blending」といって、学習器の出力を違う学習器の入力に使うのがより一般的に使われる

ただし、研究という分野に限ればアンサンブル学習せずに各々の精度に着目することが多いようだ

## 課題
ナイーブベイズ（NB）、決定木（DT）、K近傍法（KNN）を上の学習器同様に学習させ、それぞれについて評価する

ヒントは[hints.txt](./hints.txt)にあります。

### Tips:
> 実際にsklearnを使って実装を行いたいときは次の図を参考に選択すると良い<img src="http://scikit-learn.org/stable/_static/ml_map.png" widht="640px">

> タイタニック号のデータを使って例を示す。まずデータ数であるサンプル数を評価し50個以上あることを確認する。次に学習したい問題が数値的な予想（回帰）なのか生存死亡などのラベルを予測したいのかを見る。ここではラベルを予測するのでyes。次にすでに与えられたデータにそのラベルが付されているかを見ると与えられたいるのでこれもyes。次にデータ数が100k(=十万個）あるかどうかだが、ないのでyes。ここで線形SVMをためし、うまく行かなかったとして、TextDataが含まれるかを見てみる。今回は意図してそれを除外したのでNo。次にｋ近傍法を試してみる。それでもうまく行かなければ、SVM（線形でない）やアセンブル学習を試してみるという流れになる

> <img src='./軌跡.png' width='640px'>

# 3.学習器の特性

3章はsklearnのドキュメントである[このページ](http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html)に準拠して作成した。sklearnのドキュメントは他にも様々な学習器が紹介されている上、学習器を実装するに当たっての引数の意味から例まで至れり尽くせりである。

実はさまざまなパラメータがデフォルトとして指定されていて、この項ではそのパラメータ（**ハイパーパラメータ**）を変化させてその挙動を確かめる

### Tips:
> 機械学習を扱う上で「パラメータ」は頻発するワードだ。そのなかでも学習器自身が自動で獲得するものではなく、人の手で指定しなけばいけないものを他と区別して**ハイパーパラメータ**と呼ぶ。

ここからはデータに対する学習器の挙動の詳細を見ていくために可視化するが、タイタニック号のデータではデータの種類（次元数）が大きすぎて可視化しずらい。そこで、ランダムに作成した2次元のデータを学習させて学習器の特性を見ていく

## A. データを用意（生成データ）

まずはこれから学習するデータの形を見ていく

In [ ]:
from sklearn.datasets import make_moons, make_circles, make_classification

In [ ]:
moon_data = make_moons(noise=0.3, random_state=0)
circle_data = make_circles(noise=0.2, factor=0.5, random_state=1)
class_data = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           random_state=1, n_clusters_per_class=1)
datasets = [moon_data,
            circle_data,
            class_data
            ]
names = ['moon', 'circle', 'class']
for d,name in zip(datasets, names):
    X, t = d
    plt.scatter(X[:,0], X[:,1], c = t)
    plt.title(name)
    plt.colorbar()
    plt.show()

このように人間なら分割出来て当たり前な形を取っている。これを学習器はどのように分類するのかを見てみよう

学習器の特性を可視化するための関数（詳細を見る必要はない。コピペして使うときは上のデータを定義してからでないと使えないことに注意）

In [ ]:
def show_character_ML(clf):
    for d, name in zip(datasets, names):
        X, t = d
        clf.fit(X, t)# 学習データを学習
        x = np.linspace(X[:,0].min()-.5, X[:,0].max() +.5, 100)
        y = np.linspace(X[:,1].min()-.5, X[:,1].max() +.5, 100)
        xx, yy = np.meshgrid(x, y)
        grid = np.c_[xx.ravel(), yy.ravel()]
        if hasattr(clf, "decision_function"):
            plt.contourf(xx, yy, clf.decision_function(grid).reshape(xx.shape))
        else:
            plt.contourf(xx, yy, clf.predict_proba(grid)[:, 1].reshape(xx.shape))
        plt.scatter(X[:,0], X[:,1], c = t)
        plt.title(name)
        plt.colorbar()
        plt.show()

今回はあえてデータをtrainとtestに分割していないが、これは暗に「他のデータがどういう風に生成されるかは見ればわかるでしょ？」という人間的直感を当てにしているからである。要するに、「見れば汎化性能の良し悪しはなんとなく分かる」から問題ないのだ

## B. サポートベクトルマシン(SVM)

まずはデフォルトのままの挙動を見てみる

In [ ]:
show_character_ML(SVC(random_state=0))

### SVMの基本：カーネル関数

カーネル関数を用いない（線形なカーネル関数を使う）場合のSVM（線形SVM）の挙動

In [ ]:
show_character_ML(SVC(random_state=0, kernel='linear'))

実はSVMは超平面（2次元上の線、3次元上の面）で分割をしており、カーネル関数を通さないときは上のようなまっすぐな分離以外出来ない

<img src="http://datachemeng.com/wp-content/uploads/2017/06/SnapCrab_2017-6-23_16-29-54_No-00.png" width="640px" "SVM">

SVMではマージンと呼ばれる、分離したいデータ間の幅が最大になるように**線形**に分離する学習をする。線形にというのが、まさにまっすぐにということであり、分離がまっすぐなのは当たり前のことだ

ただ、まっすぐな分離ばかりでは多くの学習で問題があるのは明らかだろう。そこで登場したのがカーネル関数だ。
まっすぐな分離しか出来ないSVMにカーネル関数を与えることでグニャグニャした（非線形な）分離を可能にさせている。ここで、登場するのが係数gammaである

### gammaを変化させてみる

In [ ]:
for gamma in [1e-5, 1e+0]:
    print('gamma=',gamma)
    show_character_ML(SVC(random_state=0 ,gamma=gamma))

詳細は[ここ](http://scikit-learn.org/stable/modules/svm.html#svm-kernels)を見ること、gammaの増加にともなって非線形な識別が強くなっている

### Tips:
> - SVMではカーネル関数として大概RBF(ラジアル基底関数)を使えば良いということになっている。実際、sklearnの実装でも初期値はRBFを使うことになっている、RBFはガウス関数と同じ形状をしていて、事実上、無限次元にサンプル点を写像したあとに線形SVMで分離した場合と等価である
- ガウスRBF![RBF](https://cdn-ak.f.st-hatena.com/images/fotolife/Z/Zellij/20121019/20121019001213.png)

## C. ランダムフォレスト(RF)

例によって、まずはデフォルトのまま見てみる

In [ ]:
show_character_ML(RandomForestClassifier(random_state=0))

SVMと異なりカクカクした形状の分類をしているのが分かる。これはRFの用いる[決定木](http://scikit-learn.org/stable/modules/tree.html)の性質によるものだ

<img src="https://alphaimpact.jp/assets/2017/03/random_forest.png" width="640px" "RF">

RFはデータを復元抽出という無作為な抽出を行うことでデータを分割し、さらにランダムに選んだ特徴量を使って決定木を構成する。この決定木を集めて多数決を取ることで最終的な判定をしている。決定木の構成方法はここでは省くが、以下に見るように決定木はある次元についてスパッと2つに分割する操作を繰り返すため、その分割面は先に見たようにカクカクしたものになる

![決定木](http://scikit-learn.org/stable/_images/iris.svg)

決定木を構成する上で重要なハイパーパラメータの一つに木の深さの最大値max_depthを変えて、その変化を見る

In [ ]:
for dep in [1, 2, 4]:
    print('max_depth=',dep)
    show_character_ML(RandomForestClassifier(random_state=0, max_depth=dep))

max_depthが大きいほど学習データに過度に沿った形を表現していることが分かる。デフォルトではmax_depthが無限大に設定されている

実はランダムフォレストは先に紹介した**アンサンブル学習**の１つの応用例である。[決定木](http://scikit-learn.org/stable/modules/tree.html)という学習器を複数（デフォルトでは10個）作って、その多数決を取っている。ここではその決定木の深さの最大値であるmax_depthを変化させた

## D. ニューラルネット(NN)

デフォルトでの性質を見てみる

In [ ]:
show_character_ML(MLPClassifier(random_state=0))

次に、活性化関数(activation)を変えて見る

In [ ]:
for act_f in ['identity','logistic', 'tanh', 'relu']:
    print('activation=', act_f)
    show_character_ML(MLPClassifier(random_state=0, activation=act_f))

<img src="https://alphaimpact.jp/assets/2017/05/multilayer_perceptron.png", width="480px" "2 Layer Perceptron">

活性化関数とはニューラルネットのそれぞれのニューロンの出力に用いる関数のことである。詳しくは[活性化関数](https://ja.wikipedia.org/wiki/%E6%B4%BB%E6%80%A7%E5%8C%96%E9%96%A2%E6%95%B0)を参照してほしい

### Tips:
> 活性化関数logisticはsigmoid（シグモイド）とも呼ぶ。多くの文献ではlogisticではなくsigmoidを用いるのでこちらも覚えて置くほうが良い

ここで、活性化関数をそれぞれ見ていこう（[ここ](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)のactivationに書いてある）

In [ ]:
def identity(x):
    return x
def logistic(x):
    return 1/(1+np.exp(-x))
def tanh(x):
    return np.tanh(x)
def relu(x):
    _filter = x > 0
    return _filter*x
X = np.arange(-5, 5, 0.01)
for f,name in zip([identity, logistic, tanh, relu], ['identity', 'logistic', 'tanh','relu']):
    plt.plot(X, f(X))
    plt.title(name)
    plt.show()

どれも単純な形をしており、それほど理解も難しくないだろう。reluという名前はニューラルネット独特ではあるが、これは制御工学でも出て来るランプ関数にほかならない。

また、logisticやtanhは似た形を持っているが、歴史的に元は活性化関数がステップ関数を用いていたことに由来している

In [ ]:
def step(x):
    return (x > 0).astype(np.int)
X = np.arange(-5, 5, 0.01)
plt.plot(X, step(X))
plt.title('step')
plt.show()

2015年に[この論文中](https://www.nature.com/articles/nature14539)でReluを使うことが最善であると報告され、今ではRelu以外の活性化関数を使うことは少なくなってきた。（sklearnの初期値もreluを使っている）しかし、依然tanhなどの活性化関数を使う用途も存在する。[Why using sigmoid and tanh as the activation functions in LSTM or RNN is not problematic but this is not the case in other neural nets?](https://www.quora.com/Why-using-sigmoid-and-tanh-as-the-activation-functions-in-LSTM-or-RNN-is-not-problematic-but-this-is-not-the-case-in-other-neural-nets)

「なぜこの活性化関数が良いのか」という知見はまだ経験則以上のものはなく、「Reluは勾配消失を抑制するから」などとよく説明されるが、将来的にReluがこのまま使われる保証は今のところない

### Tips:
> ニューラルネットと一言で言ってもその言葉が指す学習器は無限大にある。今回実装したのはその中でも2層パーセプトロンである。ニューラルネットは知っての通り、脳のニューロンの働きを真似たところに由来しており、要はその真似さえしていればみんなニューラルネットである。だから、「AI」などと同様「ニューラルネット」という単語は世間のためのバズワードとしての側面が強い。ニューラルネットには代表的なものとして、CNN（Convolution NN)、RNN（Reccurent NN)といったもはや要素扱いのものや、それを組み合わせてできた実際の実装例であるAlexNetなどがある。
このように、ニューラルネットという言葉に惹かれて色々と勘違いしかねない状況なので、注意してほしい。

# おわりに

見ての通り、機械学習の実装はかなり簡単である。ライブラリを使ってしまえばほぼどのような学習器も数行で実装できる。（ただし、ニューラルネットに関してはChainer, TensorFlowなどのsklearnでないライブラリを用いるのがその学習の都合上普通である）しかし、では頭を空っぽにして使っていいかというとそうではない。そんな使い方を研究ですればたちまち質問攻めに合い、炎上すること間違いなしだ。（そういう学生が多くいるということだ）なんでも良いからというと変だが、何かしらの論理的な説明が必要になることは多々あるはずだ。そういうときに、この講義で得た知識が役立つはずである。

もちろん、この講義だけで機械学習の全てがわかる訳がない。機械学習の一部も一部でしかなく、より広い学習が今後必要だろう。しかし、機械学習へ何も前提知識を持たないとやれ「ドラえもん」だの「シュワルツネッガー」などが出てくる。こんな先入観はとりあえず取り払えたと思う。

今回は、「機械学習を正しく使用する知識の獲得」を目的として、学習器のさまざまな特性を眺めてきた。機械学習をつかうにあたって

- どの学習器を使うべきか
- どのようにパラメータを選択すべきか

はどの機械学習を扱う研究でも立ち会う問題である。選択の方法もまた様々あるが、つまるところココが研究者の腕の見せどころ、すなわち「勘」に依存しているのはなんとも言えないところではある。

実際、機械学習を使うにあたってはこの講義で習得した内容だけではかなり無理がある。例えば、機械学習では学習したいデータから**特徴量**と呼ばれる機械学習にとって都合の良い情報に変換する段階があるが、今回は一切触れなかった。これは、特徴量抽出が直接的に機械学習器の特性を示すものではなく、どちらかというと事務的な手続きや、それぞれの現象への物理的、あるいは経験的アプローチになるためで、これだけで分野ごとに本が出るほど内容が濃く、説明のきりがないためだ。他にも時系列を扱うような機械学習器などもあるが、それも省いた。これらは各自で学んでほしい。

最後に、この講義でわからなかったことはどんどん聞いてほしい。質問大歓迎！お待ちしてます